<a href="https://colab.research.google.com/github/B104020057/PIS2022/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 10.9 MB/s eta 0:00:00


In [2]:
from google. colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get ('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)

from flask import Flask, request,abort
from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    LocationAction,
)

app= Flask(__name__)
configuration = Configuration(access_token=userdata.get('LINE_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANELSECRET'))

@app.route("/callback", methods=['POST'])
def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  try:
    handler.handle(body, signature)
  except InvalidSignatureError:
    abort (400)
  return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

    action = event.message.text
    if action == 'confirm':
      reply = TemplateMessage(
        alt_text='這是確認視窗',
        template = ConfirmTemplate(
          text='你喜歡Pokemon嗎?',
          actions=[
             MessageAction(label="是",text="我超喜歡Pokemon!"),
             MessageAction(label="否",text="其實我非常喜歡,只是我要說不")
          ]
        )
      )
    elif action == 'carousel':
      carousel_template = CarouselTemplate(
            columns=[
                CarouselColumn(
                  thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                  title='夏日大作戰',
                  text='日本科幻浪漫電影',
                  actions=[
                    URIAction(label='維基百科',uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                    MessageAction(label='投票',text='我投夏日大作戰一票!')
                  ]
               ),
               CarouselColumn(
                   thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png',
                   title = '魯邦三世',
                   text='日本動畫動作冒險喜劇電影',
                   actions=[
                       URIAction(label='f', uri='https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96_%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E4%B9%8B%E5%9F%8E'),
                       MessageAction(label='投票',text='我投魯邦三世一票')
                   ]
               )
            ]
      )
      reply = TemplateMessage(
        alt_text='這是一個輪播視窗',
        template=carousel_template
      )
    else:
      reply = TextMessage(text='Thanks!')

    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[
                reply
            ]
        )
    )

ngrok_start()
if __name__ == "__main__":
  app.run()


SecretNotFoundError: Secret LINE_TOKEN does not exist.